In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Data Quality Auditor (DQA) - Pipeline de Rigor P2\n",
    "\n",
    "**Módulo:** P2 - Data Quality Auditor\n",
    "**Objetivo de Rigor:** Asegurar la integridad de los datos financieros. Aplicar un filtro de robustez (Z-Score) para identificar anomalías en el volumen (potenciales errores de adquisición o eventos atípicos).\n",
    "**Métrica P7:** Reducción de riesgo operativo causado por datos sucios."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import yfinance as yf\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from scipy.stats import zscore\n",
    "\n",
    "# 1. Configuraci\u00f3n\n",
    "TICKER = 'SPY'\n",
    "PERIOD = '5y'\n",
    "\n",
    "print(f\"--- Iniciando Auditor\u00eda de Calidad de Datos para {TICKER} ---\\n\")\n",
    "\n",
    "# Descarga de datos\n",
    "try:\n",
    "    data = yf.download(TICKER, period=PERIOD)\n",
    "    print(\"Descarga de datos exitosa.\")\n",
    "except Exception as e:\n",
    "    print(f\"FALLO DE ADQUISICI\u00d3N (P1): {e}\")\n",
    "    data = pd.DataFrame()\n",
    "    \n",
    "if data.empty:\n",
    "    print(\"El Dataframe est\u00e1 vac\u00edo. No se puede continuar con DQA.\")\n",
    "else:\n",
    "    print(f\"Filas y Columnas (Dataframe): {data.shape}\\n\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Chequeo de Nulos (NaNs) y Reporte de Integridad"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Contar valores faltantes por columna\n",
    "missing_values = data.isnull().sum()\n",
    "total_cells = np.product(data.shape)\n",
    "total_missing = missing_values.sum()\n",
    "\n",
    "print(\"--- Reporte de Valores Faltantes (MLOps Rigor) ---\")\n",
    "print(missing_values[missing_values > 0])\n",
    "\n",
    "if total_missing == 0:\n",
    "    print(f\"\\nRIGOR P2 CUMPLIDO: Cero NaNs ({total_missing} celdas faltantes). Integridad de datos OK.\")\n",
    "else:\n",
    "    nan_percent = (total_missing / total_cells) * 100\n",
    "    print(f\"\\nADVERTENCIA DE FRICCI\u00d3N: {total_missing} valores faltantes ({nan_percent:.4f}% del total). Se requiere imputaci\u00f3n.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Detecci\u00f3n de Outliers (Filtro Z-Score $\\sigma > 3$)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculamos el Z-Score para el Volumen Diario\n",
    "data['Volume_ZScore'] = zscore(data['Volume'])\n",
    "\n",
    "# Definir el filtro de outlier: Z-Score absoluto mayor a 3 (3 desviaciones est\u00e1ndar)\n",
    "OUTLIER_THRESHOLD = 3.0\n",
    "outliers = data[np.abs(data['Volume_ZScore']) > OUTLIER_THRESHOLD]\n",
    "\n",
    "print(f\"--- Reporte de Outliers en Volumen (Z-Score > {OUTLIER_THRESHOLD}) ---\")\n",
    "print(f\"Se encontraron {len(outliers)} d\u00edas at\u00edpicos:\\n\")\n",
    "print(outliers[['Volume', 'Volume_ZScore']].sort_values(by='Volume_ZScore', ascending=False).head())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Visualizaci\u00f3n de Outliers (Evidencia Gr\u00e1fica)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(12, 6))\n",
    "data['Volume'].plot(title=f'Volumen Diario de {TICKER} y Outliers (Z > {OUTLIER_THRESHOLD})')\n",
    "\n",
    "# Marcar los outliers en la gr\u00e1fica\n",
    "plt.scatter(outliers.index, outliers['Volume'], color='red', label='Outlier (Riesgo At\u00edpico)', zorder=5)\n",
    "\n",
    "plt.ylabel('Volumen Transado')\n",
    "plt.xlabel('Fecha')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}